<a href="https://colab.research.google.com/github/Gityosan/google-colab/blob/main/step1-ver1-1-twitterSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


- なし崩し
- 失笑
- 御の字
- 姑息
- すべからく
- 割愛
- 破天荒
- 役不足
- 確信犯


- https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-all
- https://developer.twitter.com/ja/docs/twitter-api/rate-limits#v2-limits


### インストール・初期設定等

In [ ]:
!python -V
!rm -rf sample_data/
from google.colab import drive

drive.mount("/content/drive")

Python 3.8.16
Mounted at /content/drive


In [ ]:
import requests
import os
import json
import time

In [ ]:
# Global settings
os.environ["TOKEN"] = "AAAAAAAAAAAAAAAAAAAAANV8iQEAAAAAH8KguG0Tdayki1U7cJK9Tif1mEY%3DxAy9AVQPDwYX7kj1fyP7fgfZFeFzEKD6TLTx0N70I6muHHFn8p"
target_url = "https://api.twitter.com/2/tweets/search/all" #Change to the endpoint you want to collect data from
base_dir = "drive/MyDrive/script/bert/"
if not os.path.exists("response_count.json"):
    with open("response_count.json", "w", encoding='utf-8') as f:
        json.dump({"meta":{"words":[]}}, f, ensure_ascii=False)

In [ ]:
def create_header():
  return {"Authorization": "Bearer {}".format(os.getenv("TOKEN"))}

def create_params(keyword, start_time, end_time, max_results = 10, next_token = None):
  #change params based on the endpoint you are using
  return {'query': keyword,
          'start_time': start_time,
          'end_time': end_time,
          'max_results': max_results,
          'expansions': '',
          'tweet.fields': 'id,text,created_at',
          'next_token': next_token}

def request_to_endpoint(url, headers, params):
  response = requests.request("GET", url, headers=headers, params=params)
  time.sleep(1)
  # print("Endpoint Response Code: " + str(response.status_code))
  if response.status_code != 200:
      raise Exception(response.status_code, response.text)
  return response.json()

def fetchTweet(keyword, start_time, end_time):
  rest = int(max_results)
  response = {"data": [], "size": 0}
  next_token = None
  while rest > 0:
    res = request_to_endpoint(url=target_url, headers=create_header(), params=create_params(keyword, start_time, end_time, max_results=500, next_token=next_token))
    rest -= 500
    if "meta" in res:
      response["size"] += int(res['meta']['result_count'])
      if "next_token" in res["meta"]:
        next_token = res["meta"]["next_token"]
    if "data" in res:
      response["data"].extend(res["data"])
  return response


### 主要処理

In [ ]:
# 単一キーワード取得
keyword = "失笑"
year=2014
max_results = 10000

word = keyword + " lang:ja"
start_time = str(year)+"-01-01T00:00:00.000Z"
end_time = str(year)+"-12-31T00:00:00.000Z"
with open("{}.json".format('only-'+str(keyword)+'-'+str(year)), "w", encoding='utf-8') as f:
  json.dump(fetchTweet(word, start_time, end_time), f, ensure_ascii=False)


In [ ]:
# 複数キーワード取得
keywords = ["失笑","なし崩し","なしくずし","御の字","姑息","須く","すべからく","割愛","破天荒","役不足","確信犯","炎上","草"]
years = list(range(2007,2021))
max_results = 10000


for k in keywords:
  word = k + " lang:ja"
  for y in years:
    with open("response_count.json") as response_count_json:
      response_count = json.load(response_count_json)
      if str(k) in response_count and str(y) in response_count[str(k)]: continue; pass
      start_time = str(y)+"-01-01T00:00:00.000Z"
      end_time = str(y)+"-12-31T00:00:00.000Z"
      response = fetchTweet(word, start_time, end_time)
      if str(k) in response_count:
        response_count[str(k)][str(y)]= response["size"]
      else:
        response_count['meta']['words'].append(str(k))
        response_count[str(k)] = {str(y): response["size"]}
      with open("response_count.json", "w", encoding='utf-8') as f:
        json.dump(response_count, f, ensure_ascii=False)
      with open(base_dir+"twitter-corpus/{}.json".format(str(k)+'-'+str(y)), "w", encoding='utf-8') as f:
        json.dump(response, f, ensure_ascii=False)
      print('fetch finish:'+str(k)+':'+str(y))

fetch finish:確信犯:2015
fetch finish:確信犯:2016
fetch finish:確信犯:2017
fetch finish:確信犯:2018
fetch finish:確信犯:2019
fetch finish:確信犯:2020
